In [80]:
# Import all the necessary libraries
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# Do not print warnings
warnings.filterwarnings('ignore')

##### Read the necessary datasets

In [81]:
causal_prior = pd.read_csv('data/CausalPriorFiltered.csv')
gene_status = pd.read_csv('data/OncoKBCancerGeneList.csv')
site_effects = pd.read_csv('data/SiteEffects.csv')

In [82]:
def getGeneType(geneName):
    ''' This function returns -1 -> Tumor Supressor Gene
                                1 -> Oncogene
                                0 -> For Both
                                -2 -> For Not Both
                                -3 -> For Unknown
    '''
    DownGeneLabel = pd.read_csv('data/OncoKBCancerGeneList.csv')
    a = DownGeneLabel.loc[(DownGeneLabel['HugoSymbol']) == (geneName)]

    if(a.empty):
        return -3

    isOncogene = (a['IsOncogene']).iloc[0]
    isTumorSupressor = (a['IsTumorSuppressorGene']).iloc[0]

    if(isOncogene == 'No' and isTumorSupressor == 'Yes'):
        return -1
    elif(isOncogene == 'Yes' and isTumorSupressor == 'No'):
        return 1
    elif(isOncogene == 'Yes' and isTumorSupressor == 'Yes'):
        return 0
    elif(isOncogene == 'No' and isTumorSupressor == 'No'):
        return -2


def upstream_gene_status(geneName):
    # Return if gene is neither Oncogene or Tumor Supressor
    geneType = getGeneType(geneName)
    if(geneType == 0 or geneType == -2 or geneType == -3):
        print('The gene is neither Oncogene or Tumor Supressor')
        return

    # Get upstream genes
    up_genes = causal_prior[causal_prior['Output'] == geneName]
    up_genes.reset_index(drop=True, inplace=True)

    # Decode InteractionType to 1 and -1
    up_genes['InteractCode'] = 0
    up_genes['InteractCode'] = up_genes['InteractionType'].apply(lambda x: 1 if str(x) == 'phosphorylates' or str(x) == 'acetylates' or str(x) == 'methylates' else -1)

    # Construct a new column i.e. 'Sites' splitting sites of each gene
    up_genes['Sites'] = ''
    up_genes['Sites'] = up_genes['Location'].str.split(';')

    # Extract site information of that Gene geneName
    gene_sites = site_effects[site_effects['HugoSymbol'] == geneName]
    gene_sites.reset_index(drop=True, inplace=True)

    # Take the Activating or Inhibiting status(1, -1) from gene_sites and put into up_genes['Sites']
    up_genes['SiteStatus'] = ''
    sites = {}
    for index, row in gene_sites.iterrows():
        sites[row['Location']] = row['Status']

    for index, row in up_genes.iterrows():
        temp  = []
        for site in row['Sites']:
            if(site in sites):
                temp.append(sites[site])
            else:
                temp.append(0)
        up_genes.at[index, 'SiteStatus'] = temp

    # Add another column for Tumor Suppressor or Oncogene label
    up_genes['DownGeneLabel'] = geneType

    # Now finally doing computation
    up_genes['Result'] = ''
    for index, row in up_genes.iterrows():
        temp  = []
        for site in row['SiteStatus']:
            result = int(site) * int(row['InteractCode']) * int(row['DownGeneLabel'])
            temp.append(result)

        up_genes.at[index, 'Result'] = temp

    # Labal genes type
    up_genes['RootGeneLabel'] = ''
    for index, row in up_genes.iterrows():
        up_genes.at[index, 'RootGeneLabel'] = getGeneType(row['Gene'])

    return up_genes


In [83]:
def get_information_from_down_gene(gene):
    # Lets get all the downstream gene of gene
    down_nodes = causal_prior[causal_prior['Gene'] == gene]
    down_nodes.reset_index(drop=True, inplace=True)
    
    # filter genes which is listed in OncoKBCancerGeneList
    col1 = []
    col2 = []
    for i in down_nodes['Output']:
        if(getGeneType(i) == 1 or getGeneType(i) == -1):
            col1.append(i)
            col2.append(getGeneType(i))
    down_known_gene = pd.DataFrame(list(zip(col1, col2)), columns=['Gene', 'Type'])
    
    # final step is to get status from genes
    final_result = pd.DataFrame()
    for i in down_known_gene['Gene']:
        output = upstream_gene_status(i)
        result = output.loc[output['Gene'] == gene]
        final_result = final_result.append(result, ignore_index=True)

    return final_result

In [88]:
output = pd.DataFrame()
for gene in gene_status['HugoSymbol']:
    output.append(get_information_from_down_gene(gene), ignore_index=True)

output